### **LIBRERÍAS**

In [9]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
from mlxtend.preprocessing import TransactionEncoder
import a_funciones as fn

### **CARGAR LOS DATOS**

In [101]:
conn=sql.connect('data\\db_movies') ### crear cuando no existe el nombre de cd  y para conectarse cuando sí existe.
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [102]:
cur.execute('select name from sqlite_master where type = "table"')
cur.fetchall()

[('ratings',), ('movies',)]

**Nombrar las tablas**

In [103]:
movies = pd.read_sql("SELECT * from movies", conn)
ratings = pd.read_sql("SELECT * from ratings", conn)

In [98]:
movies

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995,Comedy|Romance
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995,Comedy
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero,2017,Animation|Comedy|Fantasy
9739,193585,Flint,2017,Drama
9740,193587,Bungo Stray Dogs: Dead Apple,2018,Action|Animation


Se tiene una primera tabla *movies*, la cual contiene información sobre las películas (título, año de lanzamineto y los géneros asociados a la misma)

In [99]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


Se tiene también, una segunda tabla, llamada *ratings*, la cual contiene información asociada a las calificaciones que ha obtenido una película y la cantidad de usuarios que han calificado la película.

**Separar el año en una nueva columna**

In [106]:
query = """
SELECT 
    movieId, 
    TRIM(SUBSTR(title, 1, INSTR(title, '(') - 1)) AS title,  -- Elimina el año del título
    SUBSTR(title, INSTR(title, '(') + 1, 4) AS year,         -- Extrae el año
    genres
FROM 
    movies;
"""

movies = pd.read_sql(query, conn)

In [107]:
movies

,movieId,title,year,genres
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,1995,Adventure|Children|Fantasy
2,3,Grumpier Old Men,1995,Comedy|Romance
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance
4,5,Father of the Bride Part II,1995,Comedy
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero,2017,Animation|Comedy|Fantasy
9739,193585,Flint,2017,Drama
9740,193587,Bungo Stray Dogs: Dead Apple,2018,Action|Animation


In [17]:
'''
genres=movies["genres"].str.split("|")
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)
len(movies["genres"].unique())
'''

951

**Dummizar la columna de género, separando los carácteres contenidos en la misma.** 
Esta dummización se hace con el fin de poder analizar más fácilmente la base de datos

In [108]:
# Separar los géneros en columnas teniendo en cuenta el criterio de separación '|'
genres_dummies = movies['genres'].str.get_dummies(sep='|')

# Concatenar las columnas de géneros con el DataFrame original
movies_sep = pd.concat([movies, genres_dummies], axis=1)

In [109]:
movies_sep

,movieId,title,year,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,Adventure|Children|Fantasy,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,Comedy|Romance,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,Comedy|Drama|Romance,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II,1995,Comedy,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017,Action|Animation|Comedy|Fantasy,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9738,193583,No Game No Life: Zero,2017,Animation|Comedy|Fantasy,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9739,193585,Flint,2017,Drama,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple,2018,Action|Animation,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### **VISUALIZACIÓN DE LOS DATOS**

**Calificaciones generales**

In [132]:
cr = pd.read_sql("""SELECT
                 "rating" as rating,
                 count(*) as conteo
                 FROM ratings
                 group by "rating"
                 order by rating""", conn)

cr

,rating,conteo
0,0.5,1370
1,1.0,2811
2,1.5,1791
3,2.0,7551
4,2.5,5550
5,3.0,20047
6,3.5,13136
7,4.0,26818
8,4.5,8551
9,5.0,13211


In [128]:
pd.read_sql("select count(*) from ratings", conn)

# Definir los colores según las calificaciones
colors = []
for rating in cr['rating']:
    if rating == 0.5:
        colors.append('#264653')  
    elif 1 <= rating <= 2:
        colors.append('#fe4a49')  
    elif 2.5 <= rating <= 3.5:
        colors.append('#fed766')  
    elif 4 <= rating <= 5:
        colors.append('#009fb7')  

data  = go.Bar( x=cr.rating,y=cr.conteo, text=cr.conteo, textposition="outside", marker_color=colors)

layout = go.Layout(
    title={
        'text': "Conteo de Calificaciones",
        'y': 0.94,
        'x': 0.5, 
        'xanchor': 'center',  # Anclar el título al centro
        'yanchor': 'top'
    },
    xaxis={
        'title': 'Calificación',
        'tickvals': cr['rating']  # Asegurar que todos los valores del eje X se muestren
    }, 
    yaxis={'title': 'Cantidad'},
    width=800,   # Ancho del gráfico
    height=600   # Alto del gráfico
)

# Crear la figura y mostrar
fig = go.Figure(data=data, layout=layout)
fig.show()

### **ENSAYO WEB SCRAPING**  (Es una mouse-herramienta que nos ayudará más tarde)

In [96]:
import requests
from bs4 import BeautifulSoup
from IPython.display import display, HTML

# Iterar sobre las 10 primeras películas en el DataFrame
for index, row in movies.head(5).iterrows():
    movie_title = row['title']
    search_title = movie_title.replace(" ", "+")  # Reemplazar espacios por + para la URL

    # URL de búsqueda de TMDb
    search_url = f"https://www.themoviedb.org/search?query={search_title}"

    # Hacer la solicitud
    response = requests.get(search_url)

    # Verificar que la solicitud fue exitosa
    if response.status_code == 200:
        # Analizar el contenido HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Encontrar el primer resultado de la búsqueda
        movie_card = soup.find('div', class_='card')

        if movie_card:
            # Obtener el enlace de la película
            movie_link = movie_card.find('a')['href']
            full_movie_url = f"https://www.themoviedb.org{movie_link}"

            # Hacer una solicitud a la página de la película
            movie_response = requests.get(full_movie_url)

            if movie_response.status_code == 200:
                movie_soup = BeautifulSoup(movie_response.text, 'html.parser')

                # Encontrar el póster
                poster = movie_soup.find('img', class_='poster')
                if poster:
                    poster_url = f"https://image.tmdb.org/t/p/w500{poster['src']}"

                    # Mostrar la imagen en un tamaño más pequeño usando HTML
                    display(HTML(f'<img src="{poster_url}" width="150" style="margin: 5px;" />'))
                else:
                    print(f"No se encontró un póster para {movie_title}.")
            else:
                print(f"Error al acceder a la página de la película: {movie_response.status_code}")
        else:
            print(f"No se encontraron resultados para la búsqueda de {movie_title}.")
    else:
        print(f"Error en la búsqueda de {movie_title}: {response.status_code}")

In [129]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


**Calificaciones por usuario**

In [135]:
rating_users=pd.read_sql(''' SELECT "userId" as user_id,
                         count(*) as cnt_rat
                         FROM ratings
                         group by "userId"
                         order by cnt_rat asc
                         ''',conn)

rating_users

,user_id,cnt_rat
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
605,274,1346
606,448,1864
607,474,2108
608,599,2478


*Según un nuevo estudio, una persona promedio pasa más de 78.000 horas frente al televisor a lo largo de su vida. Los investigadores determinaron que los espectadores ven un promedio de 3.639 películas y 31.507 episodios de televisión, lo que equivale a unas asombrosas 78.705 horas de televisión por hora.*

In [140]:
ratings_list = np.arange(0.5, 5.5, 0.5)

# Contar cuántas veces calificó cada usuario
rating_counts = ratings.groupby('userId')['rating'].value_counts().unstack(fill_value=0)

# Reindexar para asegurarse de que todas las calificaciones estén presentes
rating_counts = rating_counts.reindex(columns=ratings_list, fill_value=0)

# Añadir la columna de total de calificaciones
rating_counts['Total'] = rating_counts.sum(axis=1)

rating_counts = rating_counts.sort_values(by='Total', ascending=False)

In [141]:
rating_counts

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,Total
userId,,,,,,,,,,,
414,1,40,20,398,122,658,232,903,76,248,2698
599,42,67,226,349,690,624,290,122,36,32,2478
474,9,36,37,173,103,383,580,569,159,59,2108
448,26,85,164,334,180,482,138,337,43,75,1864
274,2,15,37,118,139,246,472,243,63,11,1346
...,...,...,...,...,...,...,...,...,...,...,...
442,6,5,3,4,2,0,0,0,0,0,20
569,0,0,0,0,0,5,0,10,0,5,20
320,1,0,0,0,0,2,8,9,0,0,20
